# Entre lavouras e florestas: Otimizando recursos com sustentabilidade nas ciências agrárias
## Prof. Mayron César O. Moreira (DC/UFLA)
### Congresso Unificado da UFLA 2025
*11 de novembro de 2025*

In [10]:
lucro = {
    'soja': 1200,
    'milho': 800,
    'reserva': 0,
}

agua = {
    'soja': 6,
    'milho': 4,
    'reserva': 1,
}

custo = {
    'soja': 700,
    'milho': 400,
    'reserva': 200,
}

area_total = 100  # hectares

minimo_reserva_legal = 20 # hectares

limite_agua = 500 # m³/dia

orcamento_disponivel = 50000 # R$

In [11]:
import gurobipy as gp
from gurobipy import GRB

# 🌱 Variáveis de decisão

- $x \ge 0$: área (ha) destinada ao cultivo de soja.  
- $y \ge 0$: área (ha) destinada ao plantio de milho.  
- $z \ge 0$: área (ha) destinada à reserva legal.  

In [12]:
modelo = gp.Model("dinamica1")

In [13]:
x = modelo.addVar(name="soja", lb=0)
y = modelo.addVar(name="milho", lb=0)
z = modelo.addVar(name="reserva", lb=0)

* *Função objetivo:* Maximizar o lucro total proveniente das lavouras de soja e milho.

In [14]:
modelo.setObjective(
    lucro['soja'] * x +
    lucro['milho'] * y +
    lucro['reserva'] * z,
    GRB.MAXIMIZE
)

*Restrições:*  
1. **Área total disponível:** A soma das áreas destinadas à soja, milho e reserva legal não pode exceder 100 hectares.  
   $$ x + y + z \leq 100 $$

2. **Reserva legal:** Pelo menos 20 ha da área total deve ser destinada à reserva legal.  
   $$ z \geq 20 $$

3. **Limite de água:** O consumo total de água pelas lavouras de soja e milho não pode exceder 500 m³.  
   $$ 6x + 4y + z \leq 500 $$

4. **Limite de orçamento:** O custo total de plantio e manutenção das lavouras não pode exceder R$ 50.000.  
   $$ 700x + 400y + 200z \leq 50000 $$

In [15]:
# Restrições
modelo.addConstr(x + y + z <= area_total, "area_total")
modelo.addConstr(z >= minimo_reserva_legal, "reserva_legal")
modelo.addConstr(agua['soja'] * x + agua['milho'] * y + agua['reserva'] * z <= limite_agua, "limite_agua")
modelo.addConstr(custo['soja'] * x + custo['milho'] * y + custo['reserva'] * z <= orcamento_disponivel, "orcamento")

<gurobi.Constr *Awaiting Model Update*>

In [16]:
modelo.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-13620H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 3 columns and 10 nonzeros
Model fingerprint: 0x3430c986
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [8e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+04]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6000000e+04   3.837540e+02   0.000000e+00      0s
       2    8.2666667e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.266666667e+04


🧮 *Resultados*

In [17]:
import pandas as pd

resultados = pd.DataFrame({
    'Cultura': ['Soja', 'Milho', 'Reserva Legal'],
    'Área (ha)': [x.X, y.X, z.X],
    'Lucro por ha (R$)': [lucro['soja'], lucro['milho'], lucro['reserva']],
    'Custo por ha (R$)': [custo['soja'], custo['milho'], custo['reserva']],
    'Água por ha (m³/dia)': [agua['soja'], agua['milho'], agua['reserva']],
})  

resultados['Lucro Total (R$)'] = round(resultados['Área (ha)'] * resultados['Lucro por ha (R$)'], 2)
resultados['Custo Total (R$)'] = round(resultados['Área (ha)'] * resultados['Custo por ha (R$)'], 2)
resultados['Água Total (m³/dia)'] = round(resultados['Área (ha)'] * resultados['Água por ha (m³/dia)'], 2)
resultados


,Cultura,Área (ha),Lucro por ha (R$),Custo por ha (R$),Água por ha (m³/dia),Lucro Total (R$),Custo Total (R$),Água Total (m³/dia)
0,Soja,46.666667,1200,700,6,56000.00,32666.67,280.00
1,Milho,33.333333,800,400,4,26666.67,13333.33,133.33
2,Reserva Legal,20.000000,0,200,1,0.00,4000.00,20.00


In [18]:
restricoes = {
    'Área Total (ha)': resultados['Área (ha)'].sum(),
    'Reserva Legal Mínima (ha)': resultados.loc[resultados['Cultura'] == 'Reserva Legal', 'Área (ha)'].values[0],
    'Água Total (m³/dia)': resultados['Água Total (m³/dia)'].sum(),
    'Custo Total (R$)': resultados['Custo Total (R$)'].sum(),
}
restricoes_df = pd.DataFrame.from_dict(restricoes, orient='index', columns=['Valor'])
restricoes_df['Limite'] = [area_total, minimo_reserva_legal, limite_agua, orcamento_disponivel]
restricoes_df['Atende Restrição'] = restricoes_df['Valor'] <= restricoes_df['Limite']
restricoes_df

,Valor,Limite,Atende Restrição
Área Total (ha),100.00,100,True
Reserva Legal Mínima (ha),20.00,20,True
Água Total (m³/dia),433.33,500,True
Custo Total (R$),50000.00,50000,True
